In [1]:
import findspark
findspark.init()

import pyspark
from operator import add
from pyspark import SparkConf
from pyspark.ml.feature import NGram
from pyspark.sql.functions import col,udf
from pyspark.sql import SQLContext
from operator import add
import numpy as np
from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry
import string
from pyspark.mllib.util import MLUtils
from pyspark.ml.linalg import Vectors

from pyspark.mllib.linalg import Vectors as MLLibVectors
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT as VectorUDTML

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

as_ml = udf(lambda v: v.asML() if v is not None else None, VectorUDTML())

sc = pyspark.SparkContext('local[*]',appName="DocClassification")
sqlc = SQLContext(sc)



In [3]:
def swap(x):
    return (x[1],x[0])

strs = sc.parallelize(["This is something I really want to test.",
                       "To test something is something I really want to do.",
                       "I need to do something that I want to do which is to test something.",
                       "Training something is something that I needed to do.",
                       "I trained something and that something was what I needed to do.",
                       "Something I needed to do was to train something.",
                       "Something trained is something that needed to be done.",
                       "If you want to train something something has to be trained.",
                       "Train it like you've never trained it before."])
strs
strs = strs.zipWithIndex()\
            .map(lambda x: swap(x))\
            .flatMapValues(lambda x: x.split())\
            .mapValues(lambda x: x.lower().strip(string.punctuation))\
            .map(lambda x: ((x[0],x[1]),1))

labs = sc.parallelize([(0,1),(1,0),(2,1),(3,1),(4,0),(5,0),(6,1),]).toDF(["did","label"])
            
print(strs.collect())
labs.show()


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: java.lang.IllegalArgumentException
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.spark.util.ClosureCleaner$.getClassReader(ClosureCleaner.scala:46)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:443)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:426)
	at scala.collection.TraversableLike$WithFilter$$anonfun$foreach$1.apply(TraversableLike.scala:733)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashTable$class.foreachEntry(HashTable.scala:230)
	at scala.colle

In [ ]:
sstrs = strs.reduceByKey(add)\
            .map(lambda x: (x[0][1],(x[0][0],x[1])))
    
vocab = sstrs.keys().distinct().zipWithIndex()

sstrs = sstrs.join(vocab).map(lambda x: (x[1][0][0],x[1][1],float(x[1][0][1])))
print(sstrs.sortBy(lambda x: x[0]).collect())

sstrs = sstrs.map(lambda x: MatrixEntry(x[0],x[1],float(x[2])))
mat = CoordinateMatrix(sstrs).toIndexedRowMatrix().rows.toDF(["did","features"])
print(mat.head())
mat.show()

In [ ]:
fin = mat.join(labs,['did']).drop("did")
fin.show()


In [ ]:
result = fin.withColumn("features", as_ml("features"))

result.show()

In [ ]:

fin.rdd.collect()


In [ ]:
labelIndexer1 = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(result)
featureIndexer1 =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(result)
(trainingData, testData) = result.randomSplit([0.7, 0.3])
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])
model = pipeline.fit(trainingData)

In [ ]:
data = sqlc.read.format("libsvm").load("data/sample_libsvm_data.txt")
data.show()
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)
data.show()
